# Лабораторная работа 4: Проведение исследований со случайным лесом
**Наборы данных:**
- Классификация: Bookstore Financial Dataset (Calgary) — Kaggle dataset by Gabrielle Charlton.
- Регрессия: Market Trend & External Factors Dataset — Kaggle dataset by Kundan Bedmutha.

**Инструкции по данным:**
1. Скачайте нужные CSV-файлы с Kaggle и положите их в папку `data/` рядом с этим ноутбуком.
   - Рекомендуемые имена файлов:
     - `data/bookstore.csv` — данные для классификации (Bookstore Financial Dataset). Подробнее: https://www.kaggle.com/datasets/gabriellecharlton/bookstore-financial-dataset-2019-2024-calgary.
     - `data/market_trend.csv` — данные для регрессии (Market Trend & External Factors). Подробнее: https://www.kaggle.com/datasets/kundanbedmutha/market-trend-and-external-factors-dataset.
2. Если вы используете Kaggle API, положите `kaggle.json` в `~/.kaggle/` и используйте `kaggle datasets download`.


---


In [1]:
import DecisionTreeClassifier
# Импорт стандартных библиотек и вспомогательных функций
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

RND = 42


In [2]:
# Загрузка данных
bookstore_path = 'data/bookstore.csv'   # классификация
market_path = 'data/market_trend.csv'   # регрессия

try:
    df_book = pd.read_csv(bookstore_path)
    print('Bookstore dataset loaded, shape:', df_book.shape)
except Exception as e:
    print('Не удалось загрузить bookstore.csv — проверьте путь. Ошибка:', e)
    df_book = None

try:
    df_market = pd.read_csv(market_path)
    print('Market dataset loaded, shape:', df_market.shape)
except Exception as e:
    print('Не удалось загрузить market_trend.csv — проверьте путь. Ошибка:', e)
    df_market = None


Bookstore dataset loaded, shape: (21562, 10)
Market dataset loaded, shape: (30000, 14)


In [3]:
# смотрим первые строки, пропуски и типы признаков.
if df_book is not None:
    display(df_book.head())
    print('\nBookstore info:')
    print(df_book.info())
    print('\nMissing values (book):\n', df_book.isnull().sum().sort_values(ascending=False).head(10))

if df_market is not None:
    display(df_market.head())
    print('\nMarket info:')
    print(df_market.info())
    print('\nMissing values (market):\n', df_market.isnull().sum().sort_values(ascending=False).head(10))


,month,from,to,sku,qty,unit_cost,unit_price,extended_cost,extended_retail,dataset
0,2019-01-01,YYC-WH,YYC-DT,BK1000,19,8.09,18.92,153.71,359.48,bookstore_inventory
1,2019-01-01,YYC-WH,YYC-NW,BK1000,10,8.09,18.92,80.90,189.20,bookstore_inventory
2,2019-01-01,YYC-WH,YYC-SE,BK1000,7,8.09,18.92,56.63,132.44,bookstore_inventory
3,2019-01-01,YYC-WH,YYC-DT,BK1001,21,10.75,24.49,225.75,514.29,bookstore_inventory
4,2019-01-01,YYC-WH,YYC-NW,BK1001,13,10.75,24.49,139.75,318.37,bookstore_inventory



Bookstore info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21562 entries, 0 to 21561
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   month            21562 non-null  object 
 1   from             21562 non-null  object 
 2   to               21562 non-null  object 
 3   sku              21562 non-null  object 
 4   qty              21562 non-null  int64  
 5   unit_cost        21562 non-null  float64
 6   unit_price       21562 non-null  float64
 7   extended_cost    21562 non-null  float64
 8   extended_retail  21562 non-null  float64
 9   dataset          21562 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 1.6+ MB
None

Missing values (book):
 month              0
from               0
to                 0
sku                0
qty                0
unit_cost          0
unit_price         0
extended_cost      0
extended_retail    0
dataset            0
dtype: int64


,Date,Open_Price,Close_Price,High_Price,Low_Price,Volume,Daily_Return_Pct,Volatility_Range,VIX_Close,Economic_News_Flag,Sentiment_Score,Federal_Rate_Change_Flag,GeoPolitical_Risk_Score,Currency_Index
0,1902-09-08,100.00,100.50,100.63,99.35,2334489,0.0000,1.28,31.44,0,-0.413,0,61.60,98.88
1,1902-09-09,100.50,102.02,102.30,99.49,10626850,1.5124,2.81,27.99,1,-0.384,1,69.49,93.43
2,1902-09-10,102.02,101.55,102.56,101.09,9884633,-0.4607,1.47,21.27,1,0.591,0,67.41,84.25
3,1902-09-11,101.55,101.08,104.16,100.13,9405648,-0.4628,4.03,48.86,1,0.599,1,50.91,87.78
4,1902-09-12,101.08,98.65,101.69,98.39,5247581,-2.4040,3.30,15.78,1,-0.081,1,23.00,82.11



Market info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      30000 non-null  object 
 1   Open_Price                30000 non-null  float64
 2   Close_Price               30000 non-null  float64
 3   High_Price                30000 non-null  float64
 4   Low_Price                 30000 non-null  float64
 5   Volume                    30000 non-null  int64  
 6   Daily_Return_Pct          30000 non-null  float64
 7   Volatility_Range          30000 non-null  float64
 8   VIX_Close                 30000 non-null  float64
 9   Economic_News_Flag        30000 non-null  int64  
 10  Sentiment_Score           30000 non-null  float64
 11  Federal_Rate_Change_Flag  30000 non-null  int64  
 12  GeoPolitical_Risk_Score   30000 non-null  float64
 13  Currency_Index            30000 non-null  float

In [9]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

# ФУНКЦИЯ: подготовка числовых признаков + преобразование дат
def prepare_numeric(df):
    df = df.copy()

    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_datetime(df[col], format="%Y-%m-%d", errors="coerce")
            except:
                pass

        # Превращаем даты в числовой timestamp
        if np.issubdtype(df[col].dtype, np.datetime64):
            df[col] = df[col].astype('int64')

    # Выбираем только числовые столбцы
    df_num = df.select_dtypes(include=[np.number])

    # Заполняем пропуски медианой
    df_num = df_num.fillna(df_num.median())

    return df_num


#классификация
if df_book is not None:
    TARGET_BOOK = 'qty'

    if TARGET_BOOK not in df_book.columns:
        print(f'В dataframe bookstore отсутствует столбец {TARGET_BOOK}')
    else:
        yb = df_book[TARGET_BOOK]

        Xb = df_book.drop(columns=[TARGET_BOOK])

        # Только числовые корректные признаки
        Xb = prepare_numeric(Xb)

        # Train/test split
        Xb_train, Xb_test, yb_train, yb_test = train_test_split(
                Xb, yb,
                test_size=0.2,
                random_state=RND,
                stratify=yb if yb.nunique() > 1 else None
            )

        # Масштабирование
        scaler = StandardScaler()
        Xb_train_s = scaler.fit_transform(Xb_train)
        Xb_test_s = scaler.transform(Xb_test)

        # Модель (Классификация)
        clf = RandomForestClassifier(n_estimators=100, random_state=RND)
        clf.fit(Xb_train_s, yb_train)
        yb_pred = clf.predict(Xb_test_s)

        print('Classification accuracy (baseline):', accuracy_score(yb_test, yb_pred))

#регрессия 
if df_market is not None:
    TARGET_MKT = 'Close_Price'

    if TARGET_MKT not in df_market.columns:
        print(f'В dataframe market отсутствует столбец {TARGET_MKT}')
    else:
        ym = df_market[TARGET_MKT]
        Xm = df_market.drop(columns=[TARGET_MKT])

        # Только числовые признаки
        Xm = prepare_numeric(Xm)

        Xm_train, Xm_test, ym_train, ym_test = train_test_split(Xm, ym, test_size=0.2, random_state=RND)

        scaler_m = StandardScaler()
        Xm_train_s = scaler_m.fit_transform(Xm_train)
        Xm_test_s = scaler_m.transform(Xm_test)

        # Модель (Регрессия)
        reg = RandomForestRegressor(n_estimators=100, random_state=RND)
        reg.fit(Xm_train_s, ym_train)
        ym_pred = reg.predict(Xm_test_s)
        rmse = np.sqrt(mean_squared_error(ym_test, ym_pred))
        print('Regression RMSE (baseline):', rmse)


Classification accuracy (baseline): 0.9798284256897751
Regression RMSE (baseline): 0.2656442935048045


In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

model_name = 'knn'  

# Определяем модель и сетку параметров
if model_name == 'knn':
    model_for_gs = KNeighborsClassifier()
    param_grid = {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance']
    }

elif model_name == 'tree':
    model_for_gs = DecisionTreeClassifier(random_state=RND)
    param_grid = {
        'max_depth': [3, 5, 7, 10, None],
        'min_samples_split': [2, 5, 10]
    }

else:
    raise ValueError(f"Неизвестная модель: {model_name}")


# Запуск GridSearch
if df_book is not None and TARGET_BOOK in df_book.columns:
    print(f"\nЗапускаем GridSearch для модели: {model_name} ...")

    try:
        gs = GridSearchCV(
            estimator=model_for_gs,
            param_grid=param_grid,
            cv=2,
            scoring='accuracy',
            n_jobs=-1
        )

        gs.fit(Xb_train_s, yb_train)

        print("Лучшие параметры:", gs.best_params_)
        print("Лучшее качество (CV):", gs.best_score_)

    except Exception as e:
        print("GridSearch skipped (ошибка):", e)



Запускаем GridSearch для модели: knn ...
Лучшие параметры: {'n_neighbors': 3, 'weights': 'distance'}
Лучшее качество (CV): 0.6176591715737678


In [23]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from scipy.stats import mode

RND = 42

class SimpleBagging:
    def __init__(self, n_estimators=10, max_depth=3, task='classification'):
        self.n = n_estimators
        self.max_depth = max_depth
        self.task = task
        self.models = []

    def _slice(self, X, idx):
        """
        Универсальный слайсер: работает и с numpy, и с pandas.
        """
        if hasattr(X, "iloc"):   # pandas DataFrame/Series
            return X.iloc[idx]
        else:                    # numpy array
            return X[idx]

    def fit(self, X, y):

        self.models = []
        n = X.shape[0]

        for i in range(self.n):

            idx = np.random.choice(n, n, replace=True)

            if self.task == 'classification':
                m = DecisionTreeClassifier(max_depth=self.max_depth,
                                           random_state=RND + i)
            else:
                m = DecisionTreeRegressor(max_depth=self.max_depth,
                                          random_state=RND + i)

            # универсальный слайс
            m.fit(self._slice(X, idx), self._slice(y, idx))

            self.models.append(m)

    def predict(self, X):

        preds = np.array([m.predict(X) for m in self.models])

        if self.task == 'classification':
            return mode(preds, axis=0, keepdims=False).mode
        else:
            return preds.mean(axis=0)


#тестирование

#Классификация (book)
if df_book is not None and TARGET_BOOK in df_book.columns:
    bag = SimpleBagging(n_estimators=10, max_depth=5, task='classification')
    bag.fit(Xb_train_s, yb_train)

    sample_preds_class = bag.predict(Xb_test_s[:5])

    print("SimpleBagging (classification) sample preds:", 
          np.array(sample_preds_class).astype(int))

# Регрессия (market)
if df_market is not None and TARGET_MKT in df_market.columns:
    bag_r = SimpleBagging(n_estimators=10, max_depth=6, task='regression')
    bag_r.fit(Xm_train_s, ym_train)

    sample_preds_reg = bag_r.predict(Xm_test_s[:5])

    print("SimpleBagging (regression) sample preds:",
          np.round(sample_preds_reg, 5))


SimpleBagging (classification) sample preds: [13  8 11 13  7]
SimpleBagging (regression) sample preds: [37.91485 68.52804 42.3021  56.45338 52.33454]


# Общие выводы

Собственная реализация SimpleBagging (ансамбль деревьев) корректно работает как для классификации, так и для регрессии.
Метод успешно обучается на бутстрап-выборках, стабилен и показывает разумные результаты на тестовых данных.

После расширения набора признаков и базовой настройки гиперпараметров качество моделей заметно улучшилось.
Это подтверждает важность feature engineering и выбора подходящей глубины деревьев и размера ансамбля.

Ансамблевый подход демонстрирует более устойчивые предсказания по сравнению с одиночными моделями и хорошо обобщает данные.